# Imports and configs

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from random import gauss
from random import seed
from pandas import Series
from pandas.plotting import autocorrelation_plot
from matplotlib import pyplot
from plotly.subplots import make_subplots
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller

In [ ]:
airline_passengers_data_path = "https://storage.googleapis.com/edulabs-public-datasets/airline-passengers.csv"
shampoo_sales_data_path = "https://storage.googleapis.com/edulabs-public-datasets/shampoo-sales.csv"

#This dataset describes the minimum daily temperatures over 10 years (1981-1990) in the city Melbourne, Australia.
dayly_minimum_temp_data_path = "https://storage.googleapis.com/edulabs-public-datasets/daily-minimum-temperatures.csv"

# number of daily female births in California in 1959
dayly_total_female_births_data_path = "https://storage.googleapis.com/edulabs-public-datasets/daily-total-female-births.csv"

In [ ]:
pd.options.plotting.backend = "plotly"


# Load Data

In [ ]:
temp_df = pd.read_csv(dayly_minimum_temp_data_path, parse_dates=["Date"])
ts = temp_df.set_index("Date").squeeze()


In [ ]:
orig_ts = temp_df.set_index("Date").squeeze()

In [ ]:
ts.isna().sum()

In [ ]:
ts.plot()

# Preprocessing

### Time Series - set index frequency

In [ ]:
ts.index

In [ ]:
ts = ts.asfreq('D')

In [ ]:
ts.index

In [ ]:
ts.isna().sum()

### NaNs appeared - interpolate

In [ ]:
ts[ts.isna()]

In [ ]:
ts.loc['1984-12-29':'1985-01-02']

In [ ]:
ts.loc['1988-12-29':'1989-01-02']

In [ ]:
ts = ts.interpolate(method='linear')

In [ ]:
ts.loc['1984-12-29':'1985-01-02']

In [ ]:
ts.loc['1988-12-29':'1989-01-02']

# Check stationarity before fitting autoregression model

In [ ]:
result = adfuller(ts)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
for key, value in result[4].items():
    print(f"Critical Value ({key}): {value}")

In [ ]:
ts.diff(365).plot()

In [ ]:
deseasonalized = ts.diff(365).dropna()

In [ ]:
deseasonalized.shape

In [ ]:
# we'll need this later
diffs = (ts - ts.diff(365)).dropna()

In [ ]:
diffs.shape

In [ ]:
diffs.head()

In [ ]:
deseasonalized.head()

In [ ]:
plot_acf(deseasonalized, lags=50) # Adjust lags as needed
plt.show()

In [ ]:
plot_pacf(deseasonalized, lags=50) # Adjust lags as needed
plt.show()

we should take p=1 or p=2

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
lb_test = acorr_ljungbox(deseasonalized, lags=10, return_df=True)
print(lb_test)

# Find *p* (lag) value

In [ ]:
plot_acf(deseasonalized, lags=50) # Adjust lags as needed
plt.show()

In [ ]:
plot_pacf(deseasonalized, lags=50)
plt.show()

# Data split - lets use holdout strategy

In [ ]:
ts = deseasonalized

In [ ]:
train = ts[:int(0.8*(ts.shape[0]))]
test = ts[int(0.8*(ts.shape[0])):]

In [ ]:
train.shape, test.shape

# Fitting autoregression model

In [ ]:
model = AutoReg(train, lags=1)
model_fit = model.fit()


# Single time step forecasting

### In-sample prediction (predicting on train data)

In [ ]:
pred = model_fit.predict(start=0, end=1)
pred

But wait, we've just predicted value after removing seasonality - lets add it back.

The value we are looking for is seasonality diff 365 days before this date

In [ ]:
actual_pred = pred + diffs[:2]
actual_pred

In [ ]:
train.loc['1988-01-01']

In [ ]:
model_fit.predict(start='1988-01-01', end='1988-01-01') + diffs.loc['1988-01-01']

### Hold-out prediciton

In [ ]:
train

In [ ]:
model_fit.predict(start='1989-05-01', end='1989-05-01')

# Recursive Forecasting

In [ ]:
len(train)

In [ ]:
model_fit.predict(start=2626, end=2630)

In [ ]:

# Forecast next t points
t = 20
forecast = model_fit.predict(start=len(train), end=len(train)+t)
forecast

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=test[:t+1], mode='lines+markers', name='Actual'))
fig.add_trace(go.Scatter(y=forecast, mode='lines+markers', name='Predicted'))
fig.update_layout(title='Actual vs Predicted Values', xaxis_title='Date', yaxis_title='Value')
fig.show()


# Rolling Forecast

In [ ]:
train.shape

In [ ]:
diffs[:len(train)].shape

In [ ]:
# Rolling Forecast

lags = 1

history = pd.Series(train)
predictions = pd.Series()

for i in test.index:
    # note - retraining model each time
    model = AutoReg(history, lags=lags)
    model_fit = model.fit()
    yhat = model_fit.predict(start=i, end=i)

    # appending prediciton
    predictions.loc[i] = yhat.loc[i]

    # appending true value
    history.loc[i] = test.loc[i]

fig = go.Figure()
fig.add_trace(go.Scatter(y=history[len(train):] + diffs[len(train):], mode='lines+markers', name='Actual'))
fig.add_trace(go.Scatter(y=predictions + diffs[len(train):], mode='lines+markers', name='Predicted'))
fig.update_layout(title='Rolling Forecast', xaxis_title='Date', yaxis_title='Value')
fig.show()


# Validate residuals distribution and check error

In [ ]:
residuals = test - predictions

In [ ]:
residuals.plot(kind='hist', bins=50)

In [ ]:
from sklearn import metrics

print("MAE: ", metrics.mean_absolute_error(test, predictions))

add diffs

In [ ]:
test1 = test + diffs[len(train):]
predictions1 = predictions + diffs[len(train):]
residuals1 = test1 - predictions1
residuals1.plot(kind='hist', bins=50)

In [ ]:
print("MAE: ", metrics.mean_absolute_error(test1, predictions1))

In [ ]:
print("MAPE: ", metrics.mean_absolute_percentage_error(test1, predictions1))

### Calc train (in-sample) errors

In [ ]:
# train error rolling forecast
lags = 1
model = AutoReg(train, lags=1)
model_fit = model.fit()

predictions = pd.Series()

for i in range(1, len(train)):
    next_ts = train.index[i]
    yhat = model_fit.predict(start=next_ts, end=next_ts)

    # appending prediciton
    predictions.loc[next_ts] = yhat.iloc[0]



fig = go.Figure()
fig.add_trace(go.Scatter(y=train, mode='lines+markers', name='Actual'))
fig.add_trace(go.Scatter(y=predictions, mode='lines+markers', name='Predicted'))
fig.update_layout(title='Rolling Forecast', xaxis_title='Date', yaxis_title='Value')
fig.show()

In [ ]:
from sklearn import metrics

print("Train MAE: ", metrics.mean_absolute_error(train[1:], predictions))


# Exercise 1

1. Use births dataset, and train autoregression model to predict female births for the upcoming week (7 days)
2. Perform stationarity check before fitting the model
3. Select p (lags) to feed into the model
4. Use holdout strategy for data split (split 85% / 15%)
5. Try Recursive forecasting, plot actual vs predicted, print MAE
6. Try Rolling forecasting, plot actual vs predicted, print MAE. But this time you have the following limitation - you know that in the production you don't get new births data every day, you get reports every 2 days. Write rolling forecasting taking this limitation into account

In [ ]:
# number of daily female births in California in 1959
dayly_total_female_births_data_path = "https://storage.googleapis.com/edulabs-public-datasets/daily-total-female-births.csv"
births_df = pd.read_csv(dayly_total_female_births_data_path, parse_dates=["Date"])